## Default usage

In [1]:
from connectors.postgres_connector import PostgreSQLConnector
# For PostgreSQL
pg_credentials = {
    'dbname': 'postgres',
    'user': 'postgres',
    'password': 'mysecretpassword',
    'host': 'localhost',  # or another host address
    'port': 5432  # default PostgreSQL port
}

pg_connector = PostgreSQLConnector(pg_credentials)
pg_connector.connect()

In [2]:
from query_translator.translator import MixtralQueryTranslator

url = "http://localhost:1234/v1/chat/completions"
api_key=""

query_translator = MixtralQueryTranslator(llm_api_key=api_key, 
                                          llm_endpoint_url=url, 
                                          db_connector=pg_connector,
                                          llm_offline=False)
#database_ddl = query_translator.enrich_ddl_with_comments(pg_connector.get_all_schemas_ddl())

========>  True
========>  False
========>  False


In [3]:
question = "What are the suppliers with products that have the most reviews from costumers ?"

answer = query_translator.answer(question)
print(answer)

Using cached ddl...
 To find the suppliers with products that have the most reviews from customers, we first need to join the necessary tables (suppliers, products, and customerfeedback) and count the number of reviews each product has received. Then, we can join this result with the products table to get the supplier information and finally find the suppliers with the most reviews by using the aggregate function `MAX()`. Here's the SQL query:
```sql
WITH product_reviews AS (
  SELECT
    p.supplierid,
    p.productid,
    COUNT(cf.feedbackid) AS num_reviews
  FROM
    products p
    JOIN customerfeedback cf ON p.productid = cf.productid
  GROUP BY
    p.supplierid,
    p.productid
),
supplier_max_reviews AS (
  SELECT
    supplierid,
    MAX(num_reviews) AS max_reviews
  FROM
    product_reviews
  GROUP BY
    supplierid
)
SELECT
  s.supplierid,
  s.suppliername
FROM
  suppliers s
  JOIN supplier_max_reviews smr ON s.supplierid = smr.supplierid
WHERE
  smr.max_reviews = (
    SELECT
 

/Users/mehditantaoui/Documents/Challenges/GenAI-SQL/NaturalQuery/src/connectors/base_connector.py:33: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(query, self.connection)


 The suppliers with products that have the most reviews from customers are "Quality Goods Inc." and "Global Supplies". Based on the query results, these suppliers have the highest number of reviews for their products.


## Query in a different language

In [5]:
query_translator = MixtralQueryTranslator(
    llm_api_key=api_key, llm_endpoint_url=url, 
    db_connector=pg_connector,
    language='Fr', 
    llm_offline=False)


========>  True
========>  False
========>  False


In [6]:
question = "Quels sont les fournisseurs qui ont des produits les plus notés par les clients ?"

answer = query_translator.answer(question)
print(answer)

Using cached ddl...
 Sure, I can help you with that! Here is the SQL code snippet for your question:

Suppose we want to find the top 3 categories with the highest average rating from customer feedback.

We can achieve this by using the following SQL query:
```sql
SELECT c1.categoryname, AVG(cf1.rating) as avg_rating
FROM categories c1
JOIN products p1 ON c1.categoryid = p1.categoryid
JOIN customerfeedback cf1 ON p1.productid = cf1.productid
GROUP BY c1.categoryname
ORDER BY avg_rating DESC
LIMIT 3;
```
Explanation:

1. We start by joining the `categories`, `products`, and `customerfeedback` tables based on their foreign keys.
2. Then, we group the results by the `categoryname` column to calculate the average rating for each category.
3. We order the results by the average rating in descending order to get the top 3 categories.
4. Finally, we limit the results to the top 3 categories using the `LIMIT` keyword.

Note: This query assumes that there is at least one piece of feedback for e

/Users/mehditantaoui/Documents/Challenges/GenAI-SQL/NaturalQuery/src/connectors/base_connector.py:33: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(query, self.connection)


 La requête sélectionne le nom de la catégorie et la note moyenne de la table "categories", qui est jointe avec les tables "products" et "customerfeedback" à l'aide des ID de catégorie et de produit en tant que clés. Les résultats sont regroupés par nom de catégorie, triés par note moyenne en ordre décroissant et limités aux 3 premières catégories ayant les notes moyennes les plus élevées. Sur la base des résultats fournis, les trois premières catégories sont Vêtements avec une note moyenne de 5,0, Électronique avec une note moyenne de 4,0 et toute autre catégorie ayant une note moyenne inférieure à 4,0.
